# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [8]:
# import all the dependencies
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from azureml.core import Environment, ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import ScriptRunConfig
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import shutil

ws = Workspace.from_config()

In [2]:
# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
# source: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute(class)?view=azure-ml-py

cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress.......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
The following project uses data from Kaggle: https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset.
This dataset contains information on default payments, demographic factors, credit data, history of payment, and bill statements of credit card clients in Taiwan from April 2005 to September 2005. The description of each column can be found on kaggle. Our task will be to create ML models that will predict whether or not someone will default their payment in the following month. 

The data is stored is stored as csv file in the current folder. We will use python SDK to upload and register the data in Azure.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'azureml-capstone'

experiment=Experiment(ws, experiment_name)

In [4]:
ds_path = "https://raw.githubusercontent.com/albertwibowo/nd00333-capstone/master/starter_file/UCI_Credit_Card.csv"
ds = TabularDatasetFactory.from_delimited_files(path=ds_path)

df = ds.to_pandas_dataframe()
df.describe()


,limit_bal,sex,education,marriage,age,pay_0,pay_2,pay_3,pay_4,pay_5,...,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default.payment.next.month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,...,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,...,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [5]:
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

key = 'credit_default_dataset'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("The dataset was found!")

else:
    url = ds_path
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(ws,key)

df = dataset.to_pandas_dataframe()
train, test = train_test_split(df, shuffle=True, random_state=42)
train.to_csv('train.csv', index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore, 'train.csv')])

Uploading an estimated of 1 files
Uploading ./train.csv
Uploaded ./train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [18]:
%%writefile conda_dependencies.yml
dependencies:
- python=3.6.4
- scikit-learn
- pandas
- numpy
- pip:
  - azureml-defaults

Overwriting conda_dependencies.yml


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [28]:
early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

# parameter sampler
param_sampling = RandomParameterSampling( {
    'n_estimators': choice(100, 200, 500),
    'learning_rate': uniform(0.1, 1.0),
    'max_depth': choice(1, 3, 5)
    }
)

sklearn_env = Environment.from_conda_specification(name="sklearn-env",file_path="./conda_dependencies.yml")

# configure and submit your training run
run_config = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            compute_target=cpu_cluster,
                            environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=run_config,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='AUC_weighted',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4)

In [29]:
#TODO: Submit your experiment
run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [31]:
best_run = run.get_best_run_by_primary_metric()
print('Best model runId: ' + str(best_run.get_details()['runId']),
      'Best model hyperparameters: ' + str(best_run.get_details()['runDefinition']['arguments']), 
      'Best model  weighted AUC: ' + str(best_run.get_metrics()['AUC_weighted']), sep = '\n\n')

Best model runId: HD_086c738f-33d5-484a-9596-021900feab2f_7

Best model hyperparameters: ['--learning_rate', '0.1429903627672755', '--max_depth', '1', '--n_estimators', '500']

Best model  weighted AUC: 0.7774262768028579


In [34]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
azureml-capstone,HD_086c738f-33d5-484a-9596-021900feab2f_7,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [32]:
#TODO: Save the best model

model = best_run.register_model(model_name='credit_hyperdrive_model', model_path='outputs/model.joblib')
model.download(target_dir='outputs_hyperdrive', exist_ok=True)

'outputs_hyperdrive/model.joblib'

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [52]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice

inference_config = InferenceConfig(entry_script="score.py", environment = sklearn_env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1,enable_app_insights = True)

In [53]:
service=Model.deploy(workspace=ws,
                    name="credit-hyper-service",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)

DEBUG:azureml.ArtifactsClient.batch_create_empty_artifacts-async:False:[START]
DEBUG:azureml.ArtifactsClient:ClientBase: Calling batch_create_empty_artifacts with url /artifact/v2.0/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/providers/Microsoft.MachineLearningServices/workspaces/{workspaceName}/artifacts/batch/metadata/{origin}/{container}
DEBUG:msrest.service_client:Accept header absent and forced to application/json
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.1.1:46808
DEBUG:urllib3.connectionpool:http://127.0.1.1:46808 "GET /MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01 HTTP/1.1" 200 2018
DEBUG:msrestazure.azure_active_directory:MSI: Retrieving a token from http://127.0.1.1:46808/MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01
DEBUG:msrestazure.azure_active_directory:MSI: token retrieved
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.1.1:46808
DEBUG:u

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-11-13 16:45:34+00:00 Creating Container Registry if not exists.
2021-11-13 16:45:34+00:00 Registering the environment.
2021-11-13 16:45:34+00:00 Use the existing image.
2021-11-13 16:45:34+00:00 Generating deployment configuration.
2021-11-13 16:45:38+00:00 Submitting deployment to compute.
2021-11-13 16:45:43+00:00 Checking the status of deployment credit-hyper-service..
2021-11-13 16:47:57+00:00 Checking the status of inference endpoint credit-hyper-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [54]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)


service state: Healthy

scoring URI: 
http://139d21ba-9a16-4b61-a0c2-007cb32711d2.westeurope.azurecontainer.io/score

swagger URI: 
http://139d21ba-9a16-4b61-a0c2-007cb32711d2.westeurope.azurecontainer.io/swagger.json

http://139d21ba-9a16-4b61-a0c2-007cb32711d2.westeurope.azurecontainer.io/score
http://139d21ba-9a16-4b61-a0c2-007cb32711d2.westeurope.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [69]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.model_selection import train_test_split


def standardise_col_names(df):
    df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
    return df

def preprocess_quant_features(quant_df: pd.DataFrame, method):

    if method == 'mixmaxScaler':
        scaler = MinMaxScaler().fit(quant_df)
        trans_df = scaler.transform(quant_df)
        return trans_df

    elif method == 'standardScaler':
        scaler = StandardScaler().fit(quant_df)
        trans_df = scaler.transform(quant_df)
        return trans_df

    elif method == 'robustScaler':
        scaler = RobustScaler().fit(quant_df)
        trans_df = scaler.transform(quant_df)
        return trans_df
    elif method == 'normalisation':
        scaler = Normalizer().fit(quant_df)
        trans_df = scaler.transform(quant_df)
        return trans_df
    else:
        print('Choose one of the following: minmaxScaler, standardScaler, robustScaler or normalisation')


def preprocess_cat_features(cat_df: pd.DataFrame, drop_one = True):

    col_names = cat_df.columns
    if drop_one:
        trans_df = pd.DataFrame(pd.get_dummies(cat_df, prefix= col_names, prefix_sep = '_',
                                              columns = col_names, drop_first = True))
    else:
        trans_df = pd.DataFrame(pd.get_dummies(cat_df, prefix= col_names, prefix_sep = '_',
                                              columns = col_names))

    return trans_df

def clean_data(df):

    # standardise column names
    df = standardise_col_names(df)

    # use OHE on categorical columns
    cat_df = preprocess_cat_features(df[['sex', 'education', 'marriage', 'pay_0',
                                        'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']]).reset_index().drop('index', axis=1)

    # standardise numerical columns using robust scaler
    num_df = pd.DataFrame(preprocess_quant_features(df[[x for x in df.columns if x not in ['sex', 'education', 'marriage', 'pay_0',
                                        'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6','default.payment.next.month']]], 'robustScaler'),
                     columns=[x for x in df.columns if x not in ['sex', 'education', 'marriage', 'pay_0',
                                        'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6',
                                        'default.payment.next.month']]).reset_index().drop('index', axis=1)

    # combine both categorical and numerical
    trans_df = pd.concat([num_df, cat_df], axis=1)
    y_df = df.pop('default.payment.next.month')

    return trans_df, y_df

In [83]:
    df = pd.read_csv('./UCI_Credit_Card.csv')
    x, y = clean_data(df)

    # Split data into train and test sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [107]:
tosend = x_test[:2].values.tolist()
tosend = [tosend[0][:],tosend[1][:]]



82

In [108]:
import requests
import json

data = json.dumps({"data":tosend})
headers = {'Content-Type':'application/json'}

response = requests.post(scoring_uri,data,headers=headers)
print(response.text)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 139d21ba-9a16-4b61-a0c2-007cb32711d2.westeurope.azurecontainer.io:80
DEBUG:urllib3.connectionpool:http://139d21ba-9a16-4b61-a0c2-007cb32711d2.westeurope.azurecontainer.io:80 "POST /score HTTP/1.1" 200 6


[0, 0]


TODO: In the cell below, print the logs of the web service and delete the service

In [109]:
print('Status code:',response.status_code)

print(service.get_logs())

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.1.1:46808
DEBUG:urllib3.connectionpool:http://127.0.1.1:46808 "GET /MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01 HTTP/1.1" 200 2018
DEBUG:msrestazure.azure_active_directory:MSI: Retrieving a token from http://127.0.1.1:46808/MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01
DEBUG:msrestazure.azure_active_directory:MSI: token retrieved
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.1.1:46808
DEBUG:urllib3.connectionpool:http://127.0.1.1:46808 "GET /MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01 HTTP/1.1" 200 2018
DEBUG:msrestazure.azure_active_directory:MSI: Retrieving a token from http://127.0.1.1:46808/MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01
DEBUG:msrestazure.azure_active_directory:MSI: token retrieved
DEBUG:azureml._restclient.clientbase:ClientBase: Calling get w

Status code: 200
2021-11-13T16:47:35,119769100+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-11-13T16:47:35,121393900+00:00 - rsyslog/run 
2021-11-13T16:47:35,147059000+00:00 - iot-server/run 
2021-11-13T16:47:35,146482300+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-11-13T16:47:35,531853300+00:00 - iot-server/finish 1 0
2021-11-13T16:47:35,533568200+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (64)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 92
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-11-13 16:47:39,037 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-11-13 16:47:39,042 | root | INFO | Starting up request id generator
2021-11-13 16:47:39,042 | r

In [110]:
service.delete()
cpu_cluster.delete()


DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.1.1:46808
DEBUG:urllib3.connectionpool:http://127.0.1.1:46808 "GET /MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01 HTTP/1.1" 200 2018
DEBUG:msrestazure.azure_active_directory:MSI: Retrieving a token from http://127.0.1.1:46808/MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01
DEBUG:msrestazure.azure_active_directory:MSI: token retrieved
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.1.1:46808
DEBUG:urllib3.connectionpool:http://127.0.1.1:46808 "GET /MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01 HTTP/1.1" 200 2018
DEBUG:msrestazure.azure_active_directory:MSI: Retrieving a token from http://127.0.1.1:46808/MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01
DEBUG:msrestazure.azure_active_directory:MSI: token retrieved
DEBUG:azureml._restclient.clientbase:ClientBase: Calling post 

In [111]:
model.delete()

DEBUG:azureml.ModelsClient.delete-async:False:[START]
DEBUG:azureml.ModelsClient:ClientBase: Calling delete with url /modelmanagement/v1.0/subscriptions/{subscriptionId}/resourceGroups/{resourceGroup}/providers/Microsoft.MachineLearningServices/workspaces/{workspace}/models/{id}
DEBUG:msrest.service_client:Accept header absent and forced to application/json
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.1.1:46808
DEBUG:urllib3.connectionpool:http://127.0.1.1:46808 "GET /MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01 HTTP/1.1" 200 2018
DEBUG:msrestazure.azure_active_directory:MSI: Retrieving a token from http://127.0.1.1:46808/MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01
DEBUG:msrestazure.azure_active_directory:MSI: token retrieved
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.1.1:46808
DEBUG:urllib3.connectionpool:http://127.0.1.1:46808 "GET /MSI/auth/?resource=https://managem

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

